# Differential exprssion analysis

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # DEA 
            library(DESeq2), 
            library(IHW), # Independent Hypothesis Weighting
            library(edgeR), # For CPM 
            
            # Bioconductor tools
            library(rtracklayer), # readGFF, export
            library(Rsubread), # featureCounts
            
            # Data
            library(dplyr), 
            library(rtracklayer), 
            
            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(ComplexHeatmap), 
            library(grid) # For gpar
            
        )
    )
)

In [ ]:
options(warn=-1)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO") # You need to adjust this path. I will help. 

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Convert from ENSEML ID to MGI Symbol 

In [ ]:
gft_file <- "/research/peer/fdeckert/reference/genome/GRCm39/gencode.vM33.annotation.gtf.gz"

In [ ]:
import_convert <- function(gtf_path=gft_file) {
    
    # Load reference gtf 
    gtf <- rtracklayer::import(gtf_path)
    gtf <- as.data.frame(gtf)
    # Subset data 
    convert <- gtf[, c("gene_id", "gene_name")] 
    # Rename columns 
    colnames(convert) <- c("ensembl_gene_id", "mgi_symbol")
    # Select unique entries 
    convert <- unique(convert) 
    convert <- na.omit(convert)

    message(paste("Total number of ENSEMBL ID matched with MGI SYMBOL:", nrow(convert)))
    
    # Filter convert table for unique entries
    ensembl_gene_id_multiple <- table(convert$ensembl_gene_id)
    ensembl_gene_id_multiple <- ensembl_gene_id_multiple[ensembl_gene_id_multiple>1]
    convert <- convert[!convert$ensembl_gene_id %in% names(ensembl_gene_id_multiple), ]
    message(paste("Unique ENSEMBL ID:", nrow(convert)))

    mgi_symbol_multiple <- table(convert$mgi_symbol)
    mgi_symbol_multiple <- mgi_symbol_multiple[mgi_symbol_multiple>1]
    convert <- convert[!convert$mgi_symbol %in% names(mgi_symbol_multiple), ]
    message(paste("Unique MGI SYMBOL:", nrow(convert)))
    
    return(convert)
    
}

In [ ]:
convert <- import_convert()

In [ ]:
ensembl_to_mgi <- function(data, convert) {
     
    # Filter data and convert by shared ensemble_gene_id 
    rownames(convert) <- convert$ensembl_gene_id
    data <- data[rownames(data) %in% rownames(convert), ]
    convert <- convert[rownames(convert) %in% rownames(data), ]
    convert <- convert[rownames(data), ]

    # Convert ensemble_gene_id to mgi_symbol
    rownames(data) <- convert$mgi_symbol
    
    # Add full annotation 
    data$mgi_symbol <- convert$mgi_symbol
    data$ensembl_gene_id <- rownames(convert)
    
    return(data)
    
}

# Load pre-processed data

In [ ]:
meta <- readRDS("data/bulkRNAseq/object/meta.rds")
cnt <- readRDS("data/bulkRNAseq/object/cnt.rds")

# Differential expression analysis (DEA) 

## Helper functions

In [ ]:
# Filter gene matrix by group expression 
expression_filter <- function(cnt, group=NULL, min_count=10, min_sample=3, min_group=1) {
    
    message(paste0(capture.output(table(group)), collapse="\n"))
    
    # Print number of input genes 
    message("Number genes input genes ", nrow(cnt))
    
    # Remove zero expression genes 
    cnt <- cnt[rowSums(cnt)>0, ]
    
    # Print number of genes after zero removal 
    message("Number of genes after zero removal ", nrow(cnt))
    
    x <- cnt >= min_count
    x <- as.data.frame(t(x))
    x <- split(x, f=group)
    x <- lapply(x, colSums)
    x <- do.call(rbind, x)
    x <- x >= min_sample
    x <- x[, colSums(x) >= min_group]
    genes <- colnames(x)
        
    message("Number genes after count filter ", length(genes))
        
    return(genes)
    
}

In [ ]:
# Subset meta data by group and filter expression 
data_subset <- function(meta, cnt, group_by, group, min_count=10, min_sample=3, min_group=1) {
    
    # Select cell type and subset data 
    meta <- meta[meta[group_by]==group, ]
    cnt <- cnt[, colnames(cnt) %in% rownames(meta)]
    
    # Reset order of samples in cnt and meta
    meta <- meta[intersect(colnames(cnt), rownames(meta)), ]
    cnt <- cnt[, intersect(colnames(cnt), rownames(meta))]
    
    # Drop emtyp factor levels 
    meta <- droplevels(meta)

    # Filter genes
    genes_filter <- expression_filter(cnt, min_count=min_count, min_sample=min_sample, min_group=min_group, group=paste(meta$genotype, meta$dpi))
    cnt <- cnt[genes_filter, ]
    
    return(list(meta=meta, cnt=cnt))
    
}

In [ ]:
lrt <- function(data) {
    
    # lrt result list 
    res <- list()
    
    # LRT WT
    message("LRT WT")
    
    dds <- DESeqDataSetFromMatrix(countData=data[["cnt"]][, data[["meta"]]$genotype=="IFNAR_fl"], colData=data[["meta"]][data[["meta"]]$genotype=="IFNAR_fl", ], design=~dpi)
    dds <- DESeq(dds, test="LRT", full=~dpi, reduced=~1)
    
    res[["dpi_wt_lrt"]] <- results(dds) %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.))
    
    # Wald WT
    message("Wald WT")
    dds <- DESeqDataSetFromMatrix(countData=data[["cnt"]][, data[["meta"]]$genotype=="IFNAR_fl"], colData=data[["meta"]][data[["meta"]]$genotype=="IFNAR_fl", ], design=~dpi)
    dds <- DESeq(dds, test="Wald")
    
    res[["dpi_wt_wald"]] <- list(

        results(dds, name="dpi_D1_vs_D0", test="Wald") %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.)),
        results(dds, name="dpi_D3_vs_D0", test="Wald") %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.)),
        results(dds, name="dpi_D6_vs_D0", test="Wald") %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.))

    )
    
    # LRT IFNARKO
    message("LRT KO")
    
    dds <- DESeqDataSetFromMatrix(countData=data[["cnt"]][, data[["meta"]]$genotype=="IFNAR_fl_LysM_cre"], colData=data[["meta"]][data[["meta"]]$genotype=="IFNAR_fl_LysM_cre", ], design=~dpi)
    dds <- DESeq(dds, test="LRT", full=~dpi, reduced=~1)
    
    res[["dpi_ko_lrt"]] <- results(dds) %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.))
    
    # Wald IFNARKO
    message("Wald IFNARKO")
    
    dds <- DESeqDataSetFromMatrix(countData=data[["cnt"]][, data[["meta"]]$genotype=="IFNAR_fl_LysM_cre"], colData=data[["meta"]][data[["meta"]]$genotype=="IFNAR_fl_LysM_cre", ], design=~dpi)
    dds <- DESeq(dds, test="Wald")
    
    res[["dpi_ko_wald"]] <- list(

        results(dds, name="dpi_D1_vs_D0", test="Wald") %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.)),
        results(dds, name="dpi_D3_vs_D0", test="Wald") %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.)),
        results(dds, name="dpi_D6_vs_D0", test="Wald") %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.))

    )
    
    # LRT WT vs KO
    message("LRT WT vs KO")
    
    dds <- DESeqDataSetFromMatrix(countData=data[["cnt"]], colData=data[["meta"]], design=~genotype + dpi + genotype:dpi)
    dds <- DESeq(dds, test="LRT", full=~genotype + dpi + genotype:dpi, reduced=~genotype+dpi)
    
    res[["dpi_wt_vs_ko_lrt"]] <- results(dds) %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.))
    
    # Wald WT vs KO
    message("Wald WT vs KO")
    
    dds <- DESeqDataSetFromMatrix(countData=data[["cnt"]], colData=data[["meta"]], design=~genotype + dpi + genotype:dpi)
    dds$group <- factor(paste(dds$genotype, dds$dpi, sep = "_"))
    design(dds) <- ~ group
    dds$group <- relevel(dds$group, ref = "IFNAR_fl_D0")
    dds <- DESeq(dds, test="Wald")
    
    res[["dpi_wt_vs_ko_wald"]] <- list(
        
        results(dds, contrast = c("group", "IFNAR_fl_LysM_cre_D0", "IFNAR_fl_D0"), test="Wald") %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.)), 
        results(dds, contrast = c("group", "IFNAR_fl_LysM_cre_D1", "IFNAR_fl_D1"), test="Wald") %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.)),
        results(dds, contrast = c("group", "IFNAR_fl_LysM_cre_D3", "IFNAR_fl_D3"), test="Wald") %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.)),
        results(dds, contrast = c("group", "IFNAR_fl_LysM_cre_D6", "IFNAR_fl_D6"), test="Wald") %>% as.data.frame() %>% dplyr::mutate(ihw=ihw(pvalue ~ baseMean,  data=., alpha=0.1) %>% adj_pvalues(.))

    )
    
    return(res)
    
}

In [ ]:
response_class <- function(res_lrt, res_wald, padj_thr) {
    
    # Annotate response genes from LRT 
    res_lrt <- res_lrt %>% as.data.frame() %>% 
        dplyr::mutate(response_gene_lrt=ifelse(ihw <= padj_thr, TRUE, FALSE)) %>% 
        dplyr::rename(ihw_lrt=ihw, log2FoldChange_lrt=log2FoldChange) %>% 
        dplyr::select(ihw_lrt, log2FoldChange_lrt, response_gene_lrt) %>% 
        tibble::rownames_to_column("gene")
    
    # Prepare data to annotate response class    
    res_wald <- rbind(
        
        res[["dpi_wt_wald"]][[1]] %>% as.data.frame() %>% dplyr::mutate(dpi="D1") %>% tibble::rownames_to_column("gene"), 
        res[["dpi_wt_wald"]][[2]] %>% as.data.frame() %>% dplyr::mutate(dpi="D3") %>% tibble::rownames_to_column("gene"), 
        res[["dpi_wt_wald"]][[3]] %>% as.data.frame() %>% dplyr::mutate(dpi="D6") %>% tibble::rownames_to_column("gene")
        
    ) 
    
    res_wald <- res_wald %>% 
        dplyr::mutate(dpi=factor(dpi, levels=c("D1", "D3", "D6"))) 
    
    # Set log2FoldChanges for Wald test
    res_wald <- res_wald %>% 
        dplyr::group_by(gene) %>% 
        dplyr::mutate(response_gene_wald=ifelse(any(ihw<=padj_thr), TRUE, FALSE)) %>%
        dplyr::mutate(log2FoldChange=ifelse(response_gene_wald & ihw<=padj_thr, log2FoldChange, ifelse(ihw==min(padj), log2FoldChange, 0))) %>%  
        dplyr::ungroup() %>% as.data.frame()
    
    # Annotate response class from Wald test
    response_class_wald <- res_wald %>% 
        reshape2::dcast(., gene ~ dpi, value.var="log2FoldChange") %>%
        dplyr::mutate(response_class=NA) %>% 
        dplyr::mutate(response_class=ifelse(rowSums(.[2:4]>=0)==3, "Activated", response_class)) %>% 
        dplyr::mutate(response_class=ifelse(rowSums(.[2:4]<=0)==3, "Repressed", response_class)) %>%
        dplyr::mutate(response_class=ifelse(rowSums(.[2:4]>=0)>0 & rowSums(.[2:4]<=0)>0 & is.na(response_class), "Periodical", response_class)) %>% 
        # dplyr::mutate(response_class=ifelse(.[4, drop=FALSE]!=0, paste(response_class, "(monoton)"), paste(response_class, "(transient)"))) %>% 
        dplyr::mutate(response_class=as.character(response_class))
    
    # Combine results 
    result <- list(response_class_wald, res_wald[, c("gene", "response_gene_wald")], res_lrt[, c("gene", "response_gene_lrt")]) %>% 
        purrr::reduce(full_join, by="gene") %>% unique() 
    
    # Set response gene 
    result <- result %>% 
        dplyr::mutate(response_gene=ifelse(response_gene_lrt & response_gene_wald, TRUE, FALSE))
    
    # Set rownames to gene column and remove  
    rownames(result) <- result$gene
    result$gene <- NULL
    
    return(result)
    
}

In [ ]:
perturbation_class <- function(res_wald, rc_wt, rc_ko, padj_thr) {
    
    rc_wt <- rc_wt %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, response_class, response_gene) %>% dplyr::rename(response_class_wt=response_class, response_gene_wt=response_gene)
    rc_ko <- rc_ko %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, response_class, response_gene) %>% dplyr::rename(response_class_ko=response_class, response_gene_ko=response_gene)
    
    # Prepare data to annotate response class    
    res_wald <- rbind(
        
        res_wald[[1]] %>% as.data.frame() %>% dplyr::mutate(dpi="D0") %>% tibble::rownames_to_column("gene"),
        res_wald[[2]] %>% as.data.frame() %>% dplyr::mutate(dpi="D1") %>% tibble::rownames_to_column("gene"), 
        res_wald[[3]] %>% as.data.frame() %>% dplyr::mutate(dpi="D3") %>% tibble::rownames_to_column("gene"), 
        res_wald[[4]] %>% as.data.frame() %>% dplyr::mutate(dpi="D6") %>% tibble::rownames_to_column("gene")
        
    )
    
    res_wald <- res_wald %>% 
        dplyr::mutate(dpi=factor(dpi, levels=c("D0", "D1", "D3", "D6")))
    
    # Set log2FoldChanges for Wald test
    res_wald <- res_wald %>% 
        dplyr::group_by(gene) %>% 
        dplyr::mutate(perturbation_gene=ifelse(any(ihw<=padj_thr), TRUE, FALSE)) %>% 
        dplyr::mutate(log2FoldChange=ifelse(perturbation_gene & ihw<=padj_thr, log2FoldChange, ifelse(ihw==min(ihw), log2FoldChange, 0))) %>% 
        dplyr::ungroup() %>% as.data.frame() %>% 
        reshape2::dcast(., perturbation_gene + gene ~ dpi, value.var="log2FoldChange") 
    
    result <- list(res_wald, rc_wt, rc_ko) %>% purrr::reduce(full_join, by="gene") %>% dplyr::distinct() %>%
        dplyr::mutate(response_gene_wt=ifelse(is.na(response_gene_wt), FALSE, response_gene_wt)) %>% 
        dplyr::mutate(response_gene_ko=ifelse(is.na(response_gene_ko), FALSE, response_gene_ko)) %>% 
        relocate(perturbation_gene, .after=response_gene_ko)
    
    # Annotate perturbation 
    result <- result %>% 
        dplyr::mutate(perturbation_class=ifelse(perturbation_gene, "Perturbed", "Canonical")) %>% 
        tibble::column_to_rownames("gene")
    
    return(result)
    
}

In [ ]:
result_class <- function(res, padj_rc_thr, padj_pc_thr) {
    
    rc_wt <- response_class(res[["dpi_wt_lrt"]], res[["dpi_wt_wald"]], padj_thr=padj_rc_thr)
    rc_ko <- response_class(res[["dpi_ko_lrt"]], res[["dpi_ko_wald"]], padj_thr=padj_rc_thr)
    pc_wt_vs_ko <- perturbation_class(res[["dpi_wt_vs_ko_wald"]], rc_wt, rc_ko, padj_thr=padj_pc_thr)
    
    return(list("rc_wt"=rc_wt, "rc_ko"=rc_ko, "pc_wt_vs_ko"=pc_wt_vs_ko))
    
}

In [ ]:
hm_response_class <- function(meta, cnt, result, top=NULL, response_class_n=NULL, title=NULL) {

    # Remove failed tests and filter for response genes 
    result <- na.omit(result)
    result <- result[result$response_gene, ]    
    
    # Subset count data by genes in result 
    cnt <- cnt[rownames(cnt) %in% rownames(result), ]

    # Get count data for top results
    mat <- cnt[rownames(result), ]

    # Normalize and scale data
    mat <- edgeR::cpm(mat, normalized.lib.sizes=TRUE, log=FALSE)
    mat <- t(scale(t(mat), center=TRUE, scale=TRUE))

    # Order meta by factor levels 
    meta <- dplyr::arrange(meta, dpi, genotype)

    # Order mat by meta 
    mat <- mat[, rownames(meta)]
    
    # Subset and order mat by response class 
    result$response_class <- factor(result$response_class, levels=c("Activated", "Repressed", "Periodical"))
    
    # Color heatmap cnt
    color_ramp_mat <- c(rev(brewer.pal(11,"RdBu"))[1:5], "#ffffff", rev(brewer.pal(11,"RdBu"))[7:11])
    breaks_mat <- seq(-max(abs(mat)), max(abs(mat)), length.out=length(color_ramp_mat))
    color_function_mat <- circlize::colorRamp2(breaks_mat, color_ramp_mat) 
    
    # Top annotation group 
    top_annotation <- HeatmapAnnotation(

        df=data.frame(
            
            top_annotation_dpi=meta$dpi, 
            top_annotation_genotype=meta$genotype
            

        ), 

        annotation_legend_param=list(

            top_annotation_dpi=list(title="DPI", title_gp=gpar(fontsize=18, fontface="plain"), labels_gp=gpar(fontsize=16), grid_width=unit(6, "mm"), grid_height=unit(6, "mm")), 
            top_annotation_genotype=list(title="Infection", title_gp=gpar(fontsize=18, fontface="plain"), labels_gp=gpar(fontsize=16), grid_width=unit(6, "mm"), grid_height=unit(6, "mm"))

        ), 

        col=list(

            top_annotation_dpi=color$dpi, 
            top_annotation_genotype=color$genotype 

        ), 

        simple_anno_size=unit(6, "mm"), 
        show_annotation_name=FALSE, 
        show_legend=TRUE

    )

    hm <- ComplexHeatmap::Heatmap(

        matrix=mat, 
        name="z-score",  

        col=color_function_mat, 
        na_col="white", 

        width=unit(5*ncol(mat), "mm"), 
        height=unit(5*100, "mm"), 

        column_title=title, 
        column_title_gp=gpar(fontsize=18, fontface="bold"), 

        row_names_gp=gpar(fontsize=16, fontface="plain"), 
        column_names_gp=gpar(fontsize=18, fontface="plain"), 

        cluster_rows=TRUE, 
        cluster_row_slices =FALSE, 
        show_row_dend=FALSE,    
        row_split=result$response_class,
        row_title_gp=gpar(fontsize=18, fontface="plain"),
        row_gap=unit(5, "mm"),
        show_row_names=ifelse(is.null(top), FALSE, TRUE),
        row_names_side="left",

        cluster_columns=FALSE,
        show_column_dend=FALSE, 
        column_split=factor(meta$dpi),
        column_gap=unit(2, "mm"), 
        show_column_names=FALSE, 

        top_annotation=top_annotation, 

        rect_gp=gpar(col="white", lwd=0, alpha=1), 

        heatmap_legend_param=list(title_gp=gpar(fontsize=18, fontface="plain"), labels_gp=gpar(fontsize=16), legend_height=unit(5*6, "mm"), grid_width=unit(6, "mm")), 

        border=TRUE, 
        border_gp=gpar(col="black", lwd=unit(0.5, "mm")), 

        use_raster=FALSE

    )
    
    return(hm)
       
}

In [ ]:
hm_perturbation <- function(meta, cnt, result, title=NULL) {
    
    # Subset to genes that are resonse genes in either group 
    result <- result[result$response_gene_wt | result$response_gene_ko, ]
    
    # Subset count data by genes in result 
    cnt <- cnt[rownames(cnt) %in% rownames(result), ]
    
    # Get count data for top results
    mat <- cnt[rownames(result), ]

    # Normalize and scale data
    mat <- edgeR::cpm(mat, normalized.lib.sizes=TRUE, log=FALSE)
    mat <- t(scale(t(mat), center=TRUE, scale=TRUE))
    
    # Order meta by factor levels 
    meta <- dplyr::arrange(meta, dpi, genotype)
    
    # Order mat by meta 
    mat <- mat[, rownames(meta)]
    
    # Subset and order mat by response class 
    result$perturbation_class <- factor(result$perturbation_class, levels=c("Canonical", "Perturbed"))
    
    # Color heatmap cnt
    color_ramp_mat <- c(rev(brewer.pal(11,"RdBu"))[1:5], "#ffffff", rev(brewer.pal(11,"RdBu"))[7:11])
    breaks_mat <- seq(-max(abs(mat)), max(abs(mat)), length.out=length(color_ramp_mat))
    color_function_mat <- circlize::colorRamp2(breaks_mat, color_ramp_mat) 
    
    # Top annotation group 
    top_annotation <- HeatmapAnnotation(

        df=data.frame(
            
            top_annotation_dpi=meta$dpi, 
            top_annotation_genotype=meta$genotype
            

        ), 

        annotation_legend_param=list(

            top_annotation_dpi=list(title="DPI", title_gp=gpar(fontsize=18, fontface="plain"), labels_gp=gpar(fontsize=16), grid_width=unit(6, "mm"), grid_height=unit(6, "mm")), 
            top_annotation_genotype=list(title="Infection", title_gp=gpar(fontsize=18, fontface="plain"), labels_gp=gpar(fontsize=16), grid_width=unit(6, "mm"), grid_height=unit(6, "mm"))

        ), 

        col=list(

            top_annotation_dpi=color$dpi, 
            top_annotation_genotype=color$genotype 

        ), 

        simple_anno_size=unit(6, "mm"), 
        show_annotation_name=FALSE, 
        show_legend=TRUE

    )

    hm <- ComplexHeatmap::Heatmap(

        matrix=mat, 
        name="z-score",  

        col=color_function_mat, 
        na_col="white", 

        width=unit(5*ncol(mat), "mm"), 
        height=unit(5*100, "mm"), 

        column_title=title, 
        column_title_gp=gpar(fontsize=18, fontface="bold"), 

        row_names_gp=gpar(fontsize=16, fontface="plain"), 
        column_names_gp=gpar(fontsize=18, fontface="plain"), 

        cluster_rows=TRUE, 
        show_row_dend=FALSE,    
        row_split=result$perturbation_class,
        row_title_gp=gpar(fontsize=18, fontface="plain"),
        row_gap=unit(5, "mm"),
        show_row_names=FALSE,
        row_names_side="left",

        cluster_columns=FALSE,
        show_column_dend=FALSE, 
        column_split=factor(meta$dpi),
        column_gap=unit(2, "mm"), 
        show_column_names=FALSE, 

        top_annotation=top_annotation, 
        # right_annotation=row_annotation, 

        rect_gp=gpar(col="white", lwd=0, alpha=1), 

        heatmap_legend_param=list(title_gp=gpar(fontsize=18, fontface="plain"), labels_gp=gpar(fontsize=16), legend_height=unit(5*6, "mm"), grid_width=unit(6, "mm")), 

        border=TRUE, 
        border_gp=gpar(col="black", lwd=unit(0.5, "mm")), 

        use_raster=FALSE

    )
    
    return(hm)
       
}

In [ ]:
gsea <- function(result, category="H", subcategory=NULL) {
    
    # Get gene set
    gene_set <- msigdbr::msigdbr(species="mouse", category=category, subcategory=subcategory)
    gene_set <- split(gene_set, x=gene_set$gene_symbol, f=gene_set$gs_name)
    
    # Remove failed comparison 
    result <- na.omit(result)
    
    # Set gene names 
    result$mgi_symbol <- rownames(result)
    
    # Make ranks 
    result$ihw <- ifelse(result$ihw==0, min(result$ihw[result$ihw>0]), result$ihw)
    result$sign_log_adj_p_values <- -log10(result$ihw) * sign(result$log2FoldChange)
    
    ranks <- result$sign_log_adj_p_values
    names(ranks) <- result$mgi_symbol
    ranks <- ranks[order(ranks)]
    ranks <- rev(ranks)
    
    # Retain only pathways that overlap with result lsit
    gene_set_filter <- lapply(gene_set, function(x) {sum(result$mgi_symbol %in% x)>=1})
    gene_set <- gene_set[unlist(gene_set_filter)]

    gsea <- fgsea::fgsea(
        
        pathways=gene_set,
        stats=ranks,
        nperm=100000, 
        minSize=5,
        maxSize=500
        
    )
    
    return(gsea)
    
}

In [ ]:
gsea_plot <- function(gsea, pval_thr, title=NULL, color=c(RColorBrewer::brewer.pal(8, "Set1")[1], RColorBrewer::brewer.pal(8, "Set1")[2]), color_names=c("Pos", "Neg"), size_range=5, pathway_suffix=NULL, top=20) {
    
    # Set GSEA data frame 
    gsea <- as.data.frame(gsea)
    gsea <- na.omit(gsea) 
    
    # Set color names 
    color <- setNames(color, color_names)
    
    # Fix pathway names
    if(!is.null(pathway_suffix)) {gsea$pathway <- gsub(pathway_suffix, "", gsea$pathway)}
    gsea$pathway <- gsub("_", " ", gsea$pathway)
    
    # Filter hits 
    gsea_up <- gsea[sign(gsea$NES)==+1, ]
    gsea_down <- gsea[sign(gsea$NES)==-1, ]
    
    gsea_up <- gsea_up[order(gsea_up$pval), ][1:top, ]
    gsea_down <- gsea_down[order(gsea_down$pval), ][1:top, ]
    
    gsea <- rbind(gsea_up, gsea_down)
    gsea <- na.omit(gsea)
    gsea <- distinct(gsea)
    
    # Add color 
    gsea$color <- ifelse(sign(gsea$ES)==1, names(color)[1], names(color)[2])
    gsea$color <- ifelse(gsea$pval<=pval_thr, gsea$color, NA)

    gsea$sign_log_pval_values <- -log10(gsea$pval) * sign(gsea$ES)
    
    # Order hits 
    gsea <- gsea[rev(order(gsea$sign_log_pval_values)), ]
    gsea$pathway <- factor(gsea$pathway, levels=rev(gsea$pathway))

    x_max <- max(abs(gsea$sign_log_pval_values))
    if(x_max<abs(log10(pval_thr))) {x_max <- abs(log10(pval_thr))}
    x_max <- ceiling(ceiling(x_max))
    
    # Plot 
    plot <- ggplot(gsea, aes(x=sign_log_pval_values, y=pathway, color=color)) + 
        
        geom_vline(xintercept=-log10(pval_thr), linetype="dashed") + 
        geom_vline(xintercept=log10(pval_thr), linetype="dashed") +
    
        geom_point(aes(size=abs(NES))) +

        ggtitle(title) +
        xlab("Signed -log10 adj. p-value") + ylab("") + 
        scale_x_continuous(breaks=c(-x_max, 0, x_max), limits=c(-x_max, x_max)) +
        scale_color_manual(values=color, na.value="black", drop=FALSE) +
        scale_size(range=c(0, size_range)) + 
        guides(
            
            color=guide_legend(order=1, title="Agent", keywidth=0.75, keyheight=0.75, override.aes=list(size=4)), 
            size=guide_legend(order=2, title="Abs. (NES)", keywidth=0.75, keyheight=0.75)
            
        ) +
    
        theme(
            
            legend.position="right", 
            legend.justification="top", 
            axis.text.y=element_text(size=14, hjust=1, vjust=0.5, face="plain", margin=margin(t=0, r=2, b=0, l=0), color="black")
            
        ) 
    
    return(plot)
    
}

In [ ]:
dea_vp <- function(dea, log2fc_thr=0, p_adj_thr=0.05, top_label=25, label=NULL, label_merge=FALSE, title=NULL, color_neg=RColorBrewer::brewer.pal(8, "Set1")[1], color_pos=RColorBrewer::brewer.pal(8, "Set1")[2]) {

    label <- c("Icam1", "Itga4", "Itgb1", "Vcam1", "Gdf15", "Ccl2", "Tnf", "Il6", "Tgfb1", "Stat1", "Stat2", "Spic", "Irf8", "Siglec1", "Slc40a1", "Hmox1", "Blvrb", "Treml4")
    
    # Set class
    dea <- as.data.frame(dea)
    
    # Set rownames to genes
    if("gene" %in% colnames(dea)) {rownames(dea) <- dea$gene}
    
    dea <- na.omit(dea)
    
    # Annotate entries significance by log2fc_thr and p_adj_thr
    dea$ihw <- ifelse(dea$ihw == 0, .Machine$double.xmin, dea$ihw)
    dea$sig <- ifelse(abs(dea$log2FoldChange) >= log2fc_thr & -log10(dea$ihw) >= -log10(p_adj_thr), "s", "ns")
    
    # Set color based on significance and direction of dea e.g. positive and negative 
    dea$color <- ifelse(dea$sig == "s" & dea$log2FoldChange > 0, "s_pos", "ns")
    dea$color <- ifelse(dea$sig == "s" & dea$log2FoldChange < 0, "s_neg", dea$color)
    
    color <- c(color_neg, "gray", "black", color_pos)
    names(color) <- c("s_pos", "ns", "black", "s_neg")
    
    # Create labels based log2FC and p_val_adj
    dea_pos <- dea[dea$log2FoldChange > 0 & dea$sig == "s", ]
    dea_neg <- dea[dea$log2FoldChange < 0 & dea$sig == "s", ]

    pos_labels <- dea_pos[rev(order(dea_pos$log2FoldChange)), ] %>% rownames()
    neg_labels <- dea_neg[order(dea_neg$log2FoldChange), ] %>% rownames()
    
    pos_top_labels <- dea_pos[rev(order(dea_pos$log2FoldChange)), ][1:top_label, ] %>% rownames()
    neg_top_labels <- dea_neg[order(dea_neg$log2FoldChange), ][1:top_label, ] %>% rownames()
    
    # Set labels 
    if(is.null(label)) {

        label_select <- c(pos_top_labels, neg_top_labels)
        
        
    } else if (!is.null(label) & !label_merge) {

        label_select <- label[label %in% c(pos_labels, neg_labels)]
        
    } else if (!is.null(label) & label_merge) {

        label_select <- c(label[label %in% c(pos_labels, neg_labels)], c(pos_top_labels, neg_top_labels)) %>% unique()
        
    }

    dea$label <- ifelse(rownames(dea) %in% label_select, rownames(dea), NA)

    # Plot
    vp <- ggplot(dea, aes(x=log10(baseMean), y=log2FoldChange, fill=dea$color, label=label), alpha=1) + 
    
        geom_point(size=4, shape=21, color="white") + 
        geom_hline(aes(yintercept=0), linetype="dotted", colour="black") +
        ggrepel::geom_text_repel(segment.color="black", force=20, force_pull=1, max.overlaps=getOption("ggrepel.max.overlaps", default=100), size=8, alpha=1, guide="none", segment.size=0.1, color="black") + 
        ylim(-max(abs(dea$log2FoldChange)), max(abs(dea$log2FoldChange))) + 
        ggtitle(title) + xlab("log10(base expression)") + ylab("log2FC") + 
        scale_fill_manual(values=color) + 
    
        guides(
            
            color=guide_legend(order=1, title="Group", size=2, keywidth=0.75, keyheight=0.75), 
            alpha="none"
            
        ) + 
    
        theme(

            legend.position="none", 
            aspect.ratio=1

        )
    
    return(vp)
    
}

# DEA cMo

In [ ]:
# Subset data 
data <- data_subset(meta, cnt, group_by="celltype", group="cMo", min_count=10, min_sample=3, min_group=1)

## DDS and results

In [ ]:
res <- suppressMessages(lrt(data))

## Result class

In [ ]:
resc <- result_class(res, padj_rc_thr=0.01, padj_pc_thr=0.1)

In [ ]:
options(repr.plot.width=30, repr.plot.height=25)

hm_1 <- hm_response_class(data[["meta"]][data[["meta"]]$genotype=="IFNAR_fl", ], data[["cnt"]][, data[["meta"]]$genotype=="IFNAR_fl"], resc[["rc_wt"]], response_class_n=10, title="Response genes\n cMo (IFNAR1 fl/fl)") %>% ggplotify::as.ggplot()
hm_2 <- hm_response_class(data[["meta"]][data[["meta"]]$genotype=="IFNAR_fl_LysM_cre", ], data[["cnt"]][, data[["meta"]]$genotype=="IFNAR_fl_LysM_cre"], resc[["rc_ko"]], response_class_n=10, title="Response genes\n cMo (IFNAR1 fl/fl LyzMCre wt/+)") %>% ggplotify::as.ggplot()
hm_3 <- hm_perturbation(data[["meta"]], data[["cnt"]], resc[["pc_wt_vs_ko"]], title="Perturbation genes\n cMo") %>% ggplotify::as.ggplot()

hm_1 + hm_2 + hm_3

## GSEA DPI between infection types

In [ ]:
options(repr.plot.width=4*10, repr.plot.height=12)

gsea_1 <- lapply(res[["dpi_wt_vs_ko_wald"]], function(result) {gsea(result)})
gsea_plot_1 <- lapply(1:length(gsea_1), function(i) {gsea_plot(gsea_1[[i]], pathway_suffix="HALLMARK_", color_names=c("IFNAR_fl_LysM_cre", "IFNAR_fl"), title=c("D0", "D1", "D3", "D6")[i], pval_thr=0.05)})

patchwork::wrap_plots(gsea_plot_1, ncol=4)

## Volcano DPI between infection types 

In [ ]:
options(repr.plot.width=4*9, repr.plot.height=12)

v_plot <- lapply(1:length(res[["dpi_wt_vs_ko_wald"]]), function(i) {dea_vp(res[["dpi_wt_vs_ko_wald"]][[i]], title=c("D0", "D1", "D3", "D6")[i], p_adj_thr=0.05)})
patchwork::wrap_plots(v_plot, ncol=4)

## Save results 

In [ ]:
saveRDS(list(data=data, res=res, resc=resc), "result/dea/bulkRNAseq/res_dea_cmo.rds")

In [ ]:
# Wald test genotype 
res[["dpi_wt_vs_ko_wald"]] <- lapply(res[["dpi_wt_vs_ko_wald"]], function(x) {x %>% tibble::rownames_to_column("gene")})
names(res[["dpi_wt_vs_ko_wald"]]) <- c("D0", "D1", "D3", "D6")
openxlsx::write.xlsx(res[["dpi_wt_vs_ko_wald"]], "result/dea/bulkRNAseq/res_dea_genotpye_cmo.xlsx")

# DEA PreRMP

In [ ]:
data <- data_subset(meta, cnt, group_by="celltype", group="PreRPM", min_count=10, min_sample=3, min_group=1)

## DDS and results

In [ ]:
res <- suppressMessages(lrt(data))

## Result class

In [ ]:
resc <- result_class(res, padj_rc_thr=0.01, padj_pc_thr=0.1)

In [ ]:
options(repr.plot.width=30, repr.plot.height=25)

hm_1 <- hm_response_class(data[["meta"]][data[["meta"]]$genotype=="IFNAR_fl", ], data[["cnt"]][, data[["meta"]]$genotype=="IFNAR_fl"], resc[["rc_wt"]], response_class_n=10, title="Response genes\n PreRPM (WT)") %>% ggplotify::as.ggplot()
hm_2 <- hm_response_class(data[["meta"]][data[["meta"]]$genotype=="IFNAR_fl_LysM_cre", ], data[["cnt"]][, data[["meta"]]$genotype=="IFNAR_fl_LysM_cre"], resc[["rc_ko"]], response_class_n=10, title="Response genes\n PreRPM (IFNARKO)") %>% ggplotify::as.ggplot()
hm_3 <- hm_perturbation(data[["meta"]], data[["cnt"]], resc[["pc_wt_vs_ko"]], title="Perturbation genes\n PreRPM") %>% ggplotify::as.ggplot()

hm_1 + hm_2 + hm_3

## GSEA DPI between infection types

In [ ]:
options(repr.plot.width=4*10, repr.plot.height=12)

gsea_1 <- lapply(res[["dpi_wt_vs_ko_wald"]], function(result) {gsea(result)})
gsea_plot_1 <- lapply(1:length(gsea_1), function(i) {gsea_plot(gsea_1[[i]], pathway_suffix="HALLMARK_", color_names=c("IFNAR_fl_LysM_cre", "IFNAR_fl"), title=c("D0", "D1", "D3", "D6")[i], pval_thr=0.05)})

patchwork::wrap_plots(gsea_plot_1, ncol=4)

## Volcano DPI between infection types 

In [ ]:
options(repr.plot.width=4*9, repr.plot.height=12)

v_plot <- lapply(1:length(res[["dpi_wt_vs_ko_wald"]]), function(i) {dea_vp(res[["dpi_wt_vs_ko_wald"]][[i]], title=c("D0", "D1", "D3", "D6")[i], p_adj_thr=0.05)})
patchwork::wrap_plots(v_plot, ncol=4)

## Save results 

In [ ]:
saveRDS(list(data=data, res=res, resc=resc), "result/dea/bulkRNAseq/res_dea_prerpm.rds")

In [ ]:
# Wald test genotype 
res[["dpi_wt_vs_ko_wald"]] <- lapply(res[["dpi_wt_vs_ko_wald"]], function(x) {x %>% tibble::rownames_to_column("gene")})
names(res[["dpi_wt_vs_ko_wald"]]) <- c("D0", "D1", "D3", "D6")
openxlsx::write.xlsx(res[["dpi_wt_vs_ko_wald"]], "result/dea/bulkRNAseq/res_dea_genotype_prerpm.xlsx")

# DEA RPM

In [ ]:
data <- data_subset(meta, cnt, group_by="celltype", group="RPM", min_count=10, min_sample=3, min_group=1)

## DDS and results

In [ ]:
res <- suppressMessages(lrt(data))

## Result class

In [ ]:
resc <- result_class(res, padj_rc_thr=0.01, padj_pc_thr=0.1)

In [ ]:
options(repr.plot.width=30, repr.plot.height=25)

hm_1 <- hm_response_class(data[["meta"]][data[["meta"]]$genotype=="IFNAR_fl", ], data[["cnt"]][, data[["meta"]]$genotype=="IFNAR_fl"], resc[["rc_wt"]], response_class_n=10, title="Response genes\n RPM (WT)") %>% ggplotify::as.ggplot()
hm_2 <- hm_response_class(data[["meta"]][data[["meta"]]$genotype=="IFNAR_fl_LysM_cre", ], data[["cnt"]][, data[["meta"]]$genotype=="IFNAR_fl_LysM_cre"], resc[["rc_ko"]], response_class_n=10, title="Response genes\n RPM (IFNARKO)") %>% ggplotify::as.ggplot()
hm_3 <- hm_perturbation(data[["meta"]], data[["cnt"]], resc[["pc_wt_vs_ko"]], title="Perturbation genes\n RPM") %>% ggplotify::as.ggplot()

hm_1 + hm_2 + hm_3

## GSEA DPI between infection types

In [ ]:
options(repr.plot.width=4*10, repr.plot.height=12)

gsea_1 <- lapply(res[["dpi_wt_vs_ko_wald"]], function(result) {gsea(result)})
gsea_plot_1 <- lapply(1:length(gsea_1), function(i) {gsea_plot(gsea_1[[i]], pathway_suffix="HALLMARK_", color_names=c("IFNAR_fl_LysM_cre", "IFNAR_fl"), title=c("D0", "D1", "D3", "D6")[i], pval_thr=0.05)})

patchwork::wrap_plots(gsea_plot_1, ncol=4)

## Volcano DPI between infection types 

In [ ]:
options(repr.plot.width=4*9, repr.plot.height=12)

v_plot <- lapply(1:length(res[["dpi_wt_vs_ko_wald"]]), function(i) {dea_vp(res[["dpi_wt_vs_ko_wald"]][[i]], title=c("D0", "D1", "D3", "D6")[i], p_adj_thr=0.05)})
patchwork::wrap_plots(v_plot, ncol=4)

## Save results 

In [ ]:
saveRDS(list(data=data, res=res, resc=resc), "result/dea/bulkRNAseq/res_dea_rpm.rds")

In [ ]:
# Wald test genotype 
res[["dpi_wt_vs_ko_wald"]] <- lapply(res[["dpi_wt_vs_ko_wald"]], function(x) {x %>% tibble::rownames_to_column("gene")})
names(res[["dpi_wt_vs_ko_wald"]]) <- c("D0", "D1", "D3", "D6")
openxlsx::write.xlsx(res[["dpi_wt_vs_ko_wald"]], "result/dea/bulkRNAseq/res_dea_genotype_rpm.xlsx")

# DEA EB

In [ ]:
# Subset data 
data <- data_subset(meta, cnt, group_by="celltype", group="EB", min_count=10, min_sample=3, min_group=1)

## DDS and results

In [ ]:
res <- suppressMessages(lrt(data))

## Result class

In [ ]:
resc <- result_class(res, padj_rc_thr=0.01, padj_pc_thr=0.1)

In [ ]:
options(repr.plot.width=30, repr.plot.height=25)

hm_1 <- hm_response_class(data[["meta"]][data[["meta"]]$genotype=="IFNAR_fl", ], data[["cnt"]][, data[["meta"]]$genotype=="IFNAR_fl"], resc[["rc_wt"]], response_class_n=10, title="Response genes\n cMo (WT)") %>% ggplotify::as.ggplot()
hm_2 <- hm_response_class(data[["meta"]][data[["meta"]]$genotype=="IFNAR_fl_LysM_cre", ], data[["cnt"]][, data[["meta"]]$genotype=="IFNAR_fl_LysM_cre"], resc[["rc_ko"]], response_class_n=10, title="Response genes\n cMo (IFNARKO)") %>% ggplotify::as.ggplot()
hm_3 <- hm_perturbation(data[["meta"]], data[["cnt"]], resc[["pc_wt_vs_ko"]], title="Perturbation genes\n cMo") %>% ggplotify::as.ggplot()

hm_1 + hm_2 + hm_3

## GSEA DPI between infection types

In [ ]:
options(repr.plot.width=4*10, repr.plot.height=12)

gsea_1 <- lapply(res[["dpi_wt_vs_ko_wald"]], function(result) {gsea(result)})
gsea_plot_1 <- lapply(1:length(gsea_1), function(i) {gsea_plot(gsea_1[[i]], pathway_suffix="HALLMARK_", color_names=c("IFNAR_fl_LysM_cre", "IFNAR_fl"), title=c("D0", "D1", "D3", "D6")[i], pval_thr=0.05)})

patchwork::wrap_plots(gsea_plot_1, ncol=4)

## Volcano DPI between infection types 

In [ ]:
options(repr.plot.width=4*9, repr.plot.height=12)

v_plot <- lapply(1:length(res[["dpi_wt_vs_ko_wald"]]), function(i) {dea_vp(res[["dpi_wt_vs_ko_wald"]][[i]], title=c("D0", "D1", "D3", "D6")[i], p_adj_thr=0.05)})
patchwork::wrap_plots(v_plot, ncol=4)

## Save results 

In [ ]:
saveRDS(list(data=data, res=res, resc=resc), "result/dea/bulkRNAseq/res_dea_ery.rds")

In [ ]:
# Wald test genotype 
res[["dpi_wt_vs_ko_wald"]] <- lapply(res[["dpi_wt_vs_ko_wald"]], function(x) {x %>% tibble::rownames_to_column("gene")})
names(res[["dpi_wt_vs_ko_wald"]]) <- c("D0", "D1", "D3", "D6")
openxlsx::write.xlsx(res[["dpi_wt_vs_ko_wald"]], "result/dea/bulkRNAseq/res_dea_genotype_ery.xlsx")

# Session Info

In [ ]:
sessionInfo()